### In this notebook, we annotated topics data using majority vote, with preferring gpt4o labels

In [2]:
import os
import pandas as pd
from typing import List, Dict
from rich import print
from collections import Counter

In [5]:
train_gpt4o = pd.read_csv('../../data/result/train_with_sentiment_df_llm_gpt4o.csv')
test_gpt4o = pd.read_csv('../../data/result/test_with_sentiment_df_llm_gpt4o.csv')

train_command = pd.read_csv('../../data/result/train_with_sentiment_df_llm_command_r_plus.csv')
test_command = pd.read_csv('../../data/result/test_with_sentiment_df_llm_command_r_plus.csv')

train_llama3 = pd.read_csv('../../data/result/train_with_sentiment_df_llm_llama_3.csv')
test_llama3 = pd.read_csv('../../data/result/test_with_sentiment_df_llm_llama_3.csv')

In [6]:
def majority_vote(labels_dict: Dict, labels_list: List[str], break_tie: List):
    """
    Determines the majority vote from multiple annotators' labels for a given example.

    Parameters:
    -----------
    labels_dict : dict
        A dictionary with annotator names as keys and their corresponding labels as values.
    labels_list : list
        A list containing all possible labels.
    break_tie : str
        The annotator whose label should be favored in case of a tie. If `break_tie` is not a key in `labels_dict`, its value will be preferred if it is in `labels_list`.

    Returns:
    --------
    str
        The label that represents the majority vote.
    bool
        If there is a tie or not.
    """
    labels = list(labels_dict.values())
    
    tie = False
    # break tie
    if len(set(labels)) == len(labels):
        label = labels_dict.get(break_tie, break_tie)
        tie = True
        if label not in labels_list:
            raise ValueError(f"The label '{label}' is not in the list of possible labels.")
    
    else:
        label = Counter(labels).most_common()[0][0]

    return label, tie

In [7]:
test_command.head()

,column,text,topic_id,topic_name,sentiment
0,like,أكثر ما أعجبني: المقرر ساعدني جدا في فهم نظام ...,1,دكتور المقرر,Positive
1,dislike,أكثر ما لم يعجبني: المقرر يحتوي على مواضيع ساب...,6,طريقة تدريس و تقديم المادة,Negative
2,dislike,أكثر ما لم يعجبني: أسلوب شرح المقرر,5,المتطلبات و المهام و الدرجات,Negative
3,add_suggestions,اقتراحاتي للإضافة: everything is spectacular a...,2,None,Positive
4,like,أكثر ما أعجبني: معلمة المادة متمكنة وتستجيب عل...,1,دكتور المقرر,Positive


In [8]:
# labels_list = list(train_gpt4o['topic_name'].apply(lambda x: "غير محدد" if x == 'None' else x).unique())

In [9]:
labels_list = list(test_command['sentiment'].unique())
print(labels_list)

['Positive', 'Negative', 'Neutral']

In [10]:
def get_label(label_cols: List[List[str]], break_tie: str, labels_list: List[str]):
    labels = []
    ties = []
    for i, (gpt_label, llama_label, command_label) in enumerate(zip(*label_cols)):
        labels_dict = {}
        labels_dict['gpt'] = gpt_label
        labels_dict['llama3'] = llama_label
        labels_dict['command'] = command_label

        label, tie = majority_vote(labels_dict, labels_list, break_tie=break_tie)

        labels.append(label)
        ties.append(tie)
    
    return labels, ties

In [12]:
train_gpt_labels = train_gpt4o['sentiment']
train_llama_labels = train_llama3['sentiment']
train_command_labels = train_command['sentiment']

test_gpt_labels = test_gpt4o['sentiment']
test_llama_labels = test_llama3['sentiment']
test_command_labels = test_command['sentiment']

In [13]:
train_labels, train_ties = get_label([train_gpt_labels, train_llama_labels, train_command_labels], break_tie='gpt', labels_list=labels_list)

In [14]:
test_labels, test_ties = get_label([test_gpt_labels, test_llama_labels, test_command_labels], break_tie='gpt', labels_list=labels_list)

In [15]:
test_gpt4o.head()

,column,text,topic_id,topic_name,sentiment
0,like,أكثر ما أعجبني: المقرر ساعدني جدا في فهم نظام ...,1,دكتور المقرر,Positive
1,dislike,أكثر ما لم يعجبني: المقرر يحتوي على مواضيع ساب...,6,طريقة تدريس و تقديم المادة,Negative
2,dislike,أكثر ما لم يعجبني: أسلوب شرح المقرر,5,المتطلبات و المهام و الدرجات,Negative
3,add_suggestions,اقتراحاتي للإضافة: everything is spectacular a...,2,None,Positive
4,like,أكثر ما أعجبني: معلمة المادة متمكنة وتستجيب عل...,1,دكتور المقرر,Positive


In [16]:
train_gpt4o['majority_vote'] = train_labels
train_gpt4o['tie'] = train_ties

In [17]:
test_gpt4o['majority_vote'] = test_labels
test_gpt4o['tie'] = test_ties

In [18]:
train_gpt4o.head()

,column,text,topic_id,topic_name,sentiment,majority_vote,tie
0,like,أكثر ما أعجبني: التطبيق العملي المحدث والمشابه...,3,موازنة الجزء العملي مع الجزء النظري,Positive,Positive,False
1,improve_course,اقتراحاتي للتحسين: التطبيق والبعد عن التدريس ا...,3,موازنة الجزء العملي مع الجزء النظري,Neutral,Neutral,False
2,like,أكثر ما أعجبني: the content,0,محتوى ومعلومات المقرر,Positive,Positive,False
3,improve_course,اقتراحاتي للتحسين: provide more new books,3,موازنة الجزء العملي مع الجزء النظري,Neutral,Neutral,False
4,improve_course,اقتراحاتي للتحسين: زيادة ساعات العملي,8,الوقت و الجدول,Neutral,Neutral,False


In [19]:
test_gpt4o.head()

,column,text,topic_id,topic_name,sentiment,majority_vote,tie
0,like,أكثر ما أعجبني: المقرر ساعدني جدا في فهم نظام ...,1,دكتور المقرر,Positive,Positive,False
1,dislike,أكثر ما لم يعجبني: المقرر يحتوي على مواضيع ساب...,6,طريقة تدريس و تقديم المادة,Negative,Negative,False
2,dislike,أكثر ما لم يعجبني: أسلوب شرح المقرر,5,المتطلبات و المهام و الدرجات,Negative,Negative,False
3,add_suggestions,اقتراحاتي للإضافة: everything is spectacular a...,2,None,Positive,Positive,False
4,like,أكثر ما أعجبني: معلمة المادة متمكنة وتستجيب عل...,1,دكتور المقرر,Positive,Positive,False


In [20]:
# ties len
sum(train_ties), sum(test_ties)

(7, 2)

In [21]:
train_save_path = '../../data/result/train_with_sentiment_df_majority_vote_gpt4o_preferred.csv'
if os.path.exists(train_save_path):
    print('The path exists!')
else:
    print(f'Saving to {train_save_path}...')
    train_gpt4o.to_csv(train_save_path, index=False)


test_save_path = '../../data/result/test_with_sentiment_df_majority_vote_gpt4o_preferred.csv'
if os.path.exists(test_save_path):
    print('The path exists!')
else:
    print(f'Saving to {test_save_path}...')
    test_gpt4o.to_csv(test_save_path, index=False)

Saving to ../../data/result/train_with_sentiment_df_majority_vote_gpt4o_preferred.csv...

The path exists!

In [22]:
train_gpt4o['majority_vote'].value_counts()

Positive    506
Neutral     233
Negative    212
Name: majority_vote, dtype: int64

# Convert to HF dataset

In [23]:
from datasets import Dataset, Value, ClassLabel, Features, DatasetDict, load_dataset, load_from_disk

In [24]:
import pandas as pd

train_path = '../../data/result/train_with_sentiment_df_majority_vote_gpt4o_preferred.csv'

data = pd.read_csv(train_path)

In [25]:
data.head()

,column,text,topic_id,topic_name,sentiment,majority_vote,tie
0,like,أكثر ما أعجبني: التطبيق العملي المحدث والمشابه...,3,موازنة الجزء العملي مع الجزء النظري,Positive,Positive,False
1,improve_course,اقتراحاتي للتحسين: التطبيق والبعد عن التدريس ا...,3,موازنة الجزء العملي مع الجزء النظري,Neutral,Neutral,False
2,like,أكثر ما أعجبني: the content,0,محتوى ومعلومات المقرر,Positive,Positive,False
3,improve_course,اقتراحاتي للتحسين: provide more new books,3,موازنة الجزء العملي مع الجزء النظري,Neutral,Neutral,False
4,improve_course,اقتراحاتي للتحسين: زيادة ساعات العملي,8,الوقت و الجدول,Neutral,Neutral,False


In [26]:
label_names = sorted(data['majority_vote'].unique().tolist())
label_names

['Negative', 'Neutral', 'Positive']

In [27]:
labels_list

['Positive', 'Negative', 'Neutral']

In [28]:
features = Features(
    {
        'id': Value(dtype='string'),
        'text': Value(dtype='string'),
        'labels': ClassLabel(names=label_names, num_classes=len(label_names), id=None)
    }
)

print(features)

{
    'id': Value(dtype='string', id=None),
    'text': Value(dtype='string', id=None),
    'labels': ClassLabel(names=['Negative', 'Neutral', 'Positive'], id=None)
}

In [29]:
hf_dataset = DatasetDict({
    'train': Dataset.from_dict(
        {
            'id': list(data.index.astype(str)),
            'text': list(data['text']),
            'labels': list(data['majority_vote'])
        },
        features=features
    )
})

In [30]:
hf_path = '../../data/result/sentiment_hf_dataset'

In [31]:
hf_dataset.save_to_disk(hf_path)

Saving the dataset (0/1 shards):   0%|          | 0/951 [00:00<?, ? examples/s]